In [ ]:
# !pip install llm-axe

In [ ]:
from llm_axe.agents import FunctionCaller, OnlineAgent
from llm_axe.models import OllamaChat
import time, json, ollama

# We define the functions that we want the llm to be able to call. 
# Note that documentation is not required, but should be used 
#   to help the llm for understanding what each function does.
# Specifying parameter types is optional but highly recommended
def get_time():
    return time.strftime("%I:%M %p")

def get_date():
    return time.strftime("%Y-%m-%d")

def get_location():
    return "USA"

def add(num1:int, num2:int):
    return num1 + num2

def multiply(num1:int, num2:int):
    return num1 * num2

def get_distance(lat1:int, lon1:int, lat2:int, lon2:int):
    """
    Calculates the distance between two points on the Earth's surface using the Haversine formula.
    :param lat1: latitude of point 1
    :param lon1: longitude of point 1
    :param lat2: latitude of point 2
    :param lon2: longitude of point 2
    """
    return(lat1, lon1, lat2, lon2)

In [ ]:
llm = OllamaChat(model="qwen2")
fc = FunctionCaller(llm, [get_time, get_date, get_location, add, multiply, get_distance], temperature=0)


In [ ]:
!ollama rm funcall
!ollama create funcall -f Makefuncall

In [ ]:
prompt = "I have 500 coins, I just got 200 more. How many do I have?"
new_prompt = fc.get_prompt(question=prompt)
new_prompt

In [ ]:
ollm = ollama.chat(model="funcall", messages=new_prompt, stream=True, options={
    "temperature": 0.1
})

for tex in ollm:
    print(tex['message']['content'], sep="", flush=True)

In [ ]:

# res = ollama.generate(model='openkh', prompt=prompt, stream=True, template=0)
# for text in res:
#     print(text)
result = fc.get_function(prompt)
# If no function was found, exit
if(result is not None):
    print("function found")
    func = result['function']
    params = result['parameters']
    print(func(**params))
    print(result['raw_response'])
else:
    res = ollama.generate(model='qwen2', prompt=prompt, stream=True)
    for text in res:
        print(tex['message']['content'], sep="", flush=True)


In [ ]:
ollm = ollama.chat(model="qwen2", messages=new_prompt, stream=True, options={
    "temperature": 0.1
})

for tex in ollm:
    print(tex['message']['content'], sep="", flush=True)

In [ ]:
print(func(**params))
print(result['parameters'])
print(result['prompts'])
print(result['raw_response'])


In [ ]:
prompt = "Tell me a bit about this website:  https://toscrape.com/?"
searcher = OnlineAgent(llm)
resp = searcher.search(prompt)
print(resp)

In [ ]:
def chat(text):
    q = ollama.generate(model="phi3", prompt=text, stream=True, keep_alive=-1, options={
        "temperature": 0.8
    })
    for tex in q:
        print(tex['response'], sep="", flush=True)

    print("\n\n")

In [ ]:
chats = [
    "Hello, Who are you?",
    "How are you today?"
]

for i in chats:
    chat(i)